In [1]:
%cd ../../..

/home/saplab/thaiminhpv/EFISS/AI-model-query-service


In [2]:
from transformers import ViTImageProcessor, ViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import torch
import torch.nn as nn
import faiss

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = nn.Identity()
import torch
import numpy as np
# from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import pandas as pd
df = pd.read_csv('data/shopee_crop_yolo/database.txt', sep=' ', nrows=10000, names=['img_path', 'label']).drop('label', axis=1)
df
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['img_path'],
    num_rows: 10000
})

In [3]:
# def tokenize_function(examples):
#     img = cv2.imread(examples["img_path"])
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     inputs = processor(images=img, return_tensors="pt")
#     inputs['pixel_values'] = inputs['pixel_values'].squeeze()
#     return inputs
# tokenized_dataset = dataset.map(tokenize_function, batched=False, batch_size=100, writer_batch_size=100)
# tokenized_dataset = tokenized_dataset.remove_columns(['img_path'])
# # tokenized_dataset.save_to_disk("/media/saplab/MinhNVMe/relahash/temp/tokenized_dataset.hf")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenized_dataset = tokenized_dataset.with_format("torch", device=device)
# dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=32, shuffle=False)
# model.eval()
# ...
# # logits.logits.shape

# # quantizer = faiss.IndexFlatL2(768)

# torch.save({
#     "embedding": embeddings
# }, '/media/saplab/MinhNVMe/relahash/temp/embeddings.pth')

In [4]:
embeddings = torch.load('/media/saplab/MinhNVMe/relahash/temp/embeddings_0.pt')['embedding']

FileNotFoundError: [Errno 2] No such file or directory: '/media/saplab/MinhNVMe/relahash/temp/embeddings.pth'

In [ ]:
embeddings.shape

torch.Size([10000, 768])

In [ ]:
index = faiss.IndexFlatL2(768)
index.add(embeddings.numpy())

In [ ]:
path = "/home/saplab/thaiminhpv/EFISS/AI-model-query-service/trainer/notebooks/646281ba614eb20545359b00_5_shopee_midori_m4u_crop0.jpg"
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
inputs = processor(images=img, return_tensors="pt")
inputs['pixel_values'] = inputs['pixel_values'].to(device)
inputs

{'pixel_values': tensor([[[[ 0.6627,  0.6627,  0.6627,  ...,  0.6157,  0.6157,  0.6078],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.6157,  0.6078,  0.6078],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.6157,  0.6157,  0.6078],
          ...,
          [ 0.6627,  0.6627,  0.6627,  ...,  0.7098,  0.6000,  0.4588],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.6863,  0.5922,  0.4353],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.6784,  0.5686,  0.4039]],

         [[ 0.6314,  0.6314,  0.6314,  ...,  0.6235,  0.6235,  0.6157],
          [ 0.6314,  0.6314,  0.6314,  ...,  0.6235,  0.6157,  0.6157],
          [ 0.6314,  0.6314,  0.6314,  ...,  0.6235,  0.6235,  0.6157],
          ...,
          [ 0.6627,  0.6627,  0.6627,  ...,  0.3961,  0.2706,  0.0980],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.3804,  0.2471,  0.0902],
          [ 0.6627,  0.6627,  0.6627,  ...,  0.3725,  0.2314,  0.0667]],

         [[ 0.6078,  0.6078,  0.6078,  ...,  0.5843,  0.5843,  0.5765],
          [ 0

In [ ]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
logits.shape

torch.Size([1, 768])

In [ ]:
D, I = index.search(logits.cpu().numpy(), 10)     # actual search

In [ ]:
I

array([[1097, 8920, 2417, 9370, 4875, 3611, 6287,  459, 9717, 1273]])

In [ ]:
import matplotlib.pyplot as plt

for i in I[0]:
    file_path = df['img_path'].iloc[i]
    print(file_path)
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
path = "trainer/notebooks/356916625_658549589517565_6440707211788262324_n.jpg"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)

def inference(path: str):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = torch.Tensor(img).to(device)
    inputs = processor(images=img, return_tensors="pt")
    inputs['pixel_values'] = inputs['pixel_values'].to(device)
    print(inputs['pixel_values'].shape)
    with torch.no_grad():
        logits = model(**inputs).logits
    print(logits.shape)
    D, I = index.search(logits.cpu().numpy(), 10)     # actual search

    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.title("Query image: " + path)
    plt.draw()

    plt.figure(figsize=(20, 20))
    plt.title("Result images")
    for i, e in enumerate(I[0]):
        plt.subplot(1, 10, i + 1)
        file_path = df['img_path'].iloc[e]
        print(file_path)
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
    plt.draw()
inference(path)